# *Chapter 13*<br>   Wavelet & Principal Components Analyses<br> for Nonstationary Signals & Data Compression  

| | | |
|:---:|:---:|:---:|
| ![image](Figs/Cover.png)|[From **COMPUTATIONAL PHYSICS**, 3rd Ed, 2015](http://physics.oregonstate.edu/~rubin/Books/CPbook/index.html) <br>RH Landau, MJ Paez, and CC Bordeianu (deceased) <br>Copyrights: <br> [Wiley-VCH, Berlin;](http://www.wiley-vch.de/publish/en/books/ISBN3-527-41315-4/) and [Wiley & Sons, New York](http://www.wiley.com/WileyCDA/WileyTitle/productCd-3527413154.html)<br>  R Landau, Oregon State Unv, <br>MJ Paez, Univ Antioquia,<br> C Bordeianu, Univ Bucharest, 2015.<br> Support by National Science Foundation.|![image](Figs/BackCover.png)|

**13 Wavelet & Principal Components Analyses**<br>
[13.1 Problem: Spectral Analysis of Nonstationary Signals](#13.1)<br>
[13.2 Wavelet Basics](#13.2)<br>
[13.3 Wave Packets and Uncertainty Principle (Theory)](#13.3)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[13.3.1 Wave Packet Assessment ms (Math)](#13.3.1)<br>
[13.4 Short-Time Fourier Transform (Math)](#13.4)<br>
[13.5 The Wavelet Transform](#13.5)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[13.5.1 Wavelet Basis Functions](#13.5.1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[13.5.2 Continuous Wavelet Transform](#13.5.2)<br>
[13.6 DiscreteWavelet Transforms, Multi Resolution Analysis](#13.6)<br>
&nbsp;&nbsp;&nbsp;&nbsp; [13.6.1 Pyramid Scheme Implementation](#13.6.1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[13.6.2 Daubechies Wavelets via Filtering](#13.6.2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[13.6.3 DWT Implementation and Exercise](#13.6.3)<br>
[13.7 Principal Components Analysis](#13.7)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[13.7.1 Demonstration of Principal Component Analysis](#13.7.1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[13.7.2 PCA Exercises](#13.7.2)<br> 

*There are a number of techniques that extend Fourier analysis to
signals whose forms change in time. In this chapter introduce *wavelet
analysis*, a field that has seen extensive development and application
in areas as diverse as brain waves, stock-market trends, gravitational
waves and compression of photographic images. The first part of the
chapter deals with wavelet basics, and covers the essential materials.
The next part of the chapter explores the discrete wavelet transform,
and is marked as optional as a result of its technical nature. However,
it is a beautiful bit of analysis and the basis of much of the digital
revolution. The last part of the chapter introduces principal components
analysis, another powerful tool, and especially for analyzing data whose
space and time components are correlated.*

** This Chapter’s Lecture, Slide Web Links, Applets & Animations**

| | |
|---|---|
|[All Lectures](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/index.html)|[![anything](Figs/RHLlectureMod4.png)](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/index.html)|

| *Lecture (Flash)*| *Slides* | *Sections*|*Lecture (Flash)*| *Slides* | *Sections*|  
|- - -|:- - -:|:- - -:|- - -|:- - -:|:- - -:|
|[Wavelets I](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Modules/CWavelet_I/CWavelet_I.html)| [pdf](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Slides/Slides_NoAnimate_pdf/CWT_27Nov09.pdf)|11.1|  [Wavelets II, Continuous](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Modules/CWavelet_II/CWavelet_II.html)|[pdf](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Slides/Slides_NoAnimate_pdf/DWT_15Dec09.pdf)|11.4| 
|[Wavelets III, Discrete](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Modules/DWavelet_III/DWavelet_III.html) |[pdf](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Slides/Slides_NoAnimate_pdf/DWT_15Dec09.pdf)| 11.5|  | | |


In [1]:
## 13.1  Problem: Spectral Analysis of Nonstationary Signals<a id="13.1"></a>

**Problem:** You have sampled the signal in Figure 13.1 that seems to contain
an increasing number of frequencies as time increases. Your **problem** is to
undertake a spectral analysis of this signal that tells you, in the most compact
way possible, the amount of each frequency present at each instant of time.
*Hint:* Although we want the method to be general enough to work with
numerical data, for pedagogical purposes it is useful to know that the signal is

$$\begin{align}  \tag*{13.1}
 y(t) =\begin{cases}
 \sin 2\pi t, & \mbox{for } \ 0 \leq t \leq 2,\\
 5\sin 2\pi t + 10\sin 4\pi t , & \mbox{for } \ 2 \leq t \leq 8,\\
 2.5\sin 2\pi t + 6\sin 4\pi t + 10\sin 6\pi t, &\mbox{for } \ 8 \leq t \leq
 12.
\end{cases}\end{align}$$

![image](Figs/Fig13_1.png)

**Figure 13.1** The input time signal (13.1) we wish to analyze. The signal is
seen to contain additional frequencies as time increases. The boxes are possible
placements of windows for short-time Fourier transforms.

SyntaxError: invalid syntax (<ipython-input-1-c442b2d5bb83>, line 3)

## 13.2  Wavelet Basics<a id="13.2"></a>

The Fourier analysis we used in [Chapter 12](CP12.ipynb) reveals the
amount of the harmonic functions $\sin(\omega t)$ and $\cos(\omega t)$,
and their overtones, that are present in a signal. An expansion in
periodic functions is fine for *stationary* signals (those whose forms
do not change in time) but has shortcomings for the variable form of our
**problem** signal (13.1). One such problem is that the Fourier
reconstruction has all constituent frequencies occurring simultaneously
and so does not contain *time resolution* information indicating when
each frequency occurs. Another shortcoming is that all the Fourier
components are correlated, which results in more information being
stored than may be needed to reconstruction the measured signal.

There are a number of techniques that extend simple Fourier analysis to
nonstationary signals. The idea behind wavelet analysis is to expand a
signal in a complete set of functions (wavelets), each of which
oscillates for a finite period of time, and each of which is centered at
a different time. To give you a preview before we get into the details,
we show four sample wavelets in Figure 13.2. Because each wavelet is
local in time, it is a wave packet\[*Note:* We discuss wave packets
further in §13.3.\], with its time localization leading to a spectrum
with a range of frequencies. These wave packets are called “wavelets”
because they exist for only short periods of time \[[Polimkar(01)](BiblioLinked.html#Polikar)\].

Although wavelets are required to oscillate in time, they are not
restricted to a particular functional form \[Addison(02), Goswani &
Chan(99), Gould et al.(06)\]. As a case in point, they may be
oscillating Gaussians (Morlet: top left in Figure 13.2),

$$\tag*{13.2}
\Psi(t) =    e^{2\pi i t} e^{-t^2/2\sigma^2} = (\cos 2\pi t + i \sin
2\pi t) e^{-t^2/2\sigma^2} \quad\mbox{(Morlet)},$$

the second derivative of a Gaussian (Mexican hat, top right),

$$\tag*{13.3}
\Psi(t) =- \sigma^2 \frac{d^2}{dt^2} e^{-t^2/2\sigma^2} =
\left(1-\frac{t^2}{\sigma^2}\right) e^{-t^2/2\sigma^2},$$

an up-and-down step function (lower left), or a fractal shape (bottom
right). All of these wavelets are *localized* in both time and
frequency, that is, they are large for just a limited time and contain a
finite range of frequencies. As we shall see, translating and scaling a
*mother wavelets* generates an entire set of *daughter wavelet* or basis
functions, with the daughters covering different frequency ranges at
different times.

|Figure 13.2 A Figure 13.2 B |Figure 13.2 C Figure 13.2 D| 
|:- - -:|:- - -:|
|![image](Figs/Fig13_2a.png)|![image](Figs/Fig13_2b.png)|
|![image](Figs/Fig13_2c.png)|![image](Figs/Fig13_2d.png)|

**Figure 13.2** Four possible mother wavelets that can be used to generate
entire sets of daughter wavelets. *Clockwise from top:* Morlet (real part),
Mexican hat, Daub4 e6 (explained later), and Haar. The daughter wavelets are
generated by scaling and translating these mother wavelets.

## 13.3 Packets and Uncertainty Principle (Theory)<a id="13.3"></a>

A *wave packet* or *wave train* is a collection of waves of differing
frequencies added together in such a way as to produce a pulse of width
$\Delta t$. As we shall see, the Fourier transform of a wave packet is a
pulse in the frequency domain of width $\Delta
\omega$. We will first study such wave packets analytically, and then
use others numerically. An example of a simple wave packet is just a sine wave
that oscillates at frequency $\omega_0$ for $N$ periods (Figure 13.3 A) \[[Arfken
& Weber(01)](BiblioLinked.html#arkenweber)\] 

$$\tag*{13.4}
 y(t) =\begin{cases} \sin\omega_0 t, & \mbox{for }\quad |t| < N
\frac{\pi}{\omega_0}    \equiv N \frac{T}{2},\\
0, &\mbox{for }\quad    |t|  > N \frac{\pi}{\omega_0} \equiv N
\frac{T}{2},
\end{cases}$$

where we relate the frequency to the period via the usual
$\omega_0=2\pi/T$. In terms of these parameters, the width of the wave
packet is

$$\tag*{13.5}
\Delta t = NT=N \frac{2\pi} {\omega_0}.$$

The Fourier transform of the wave packet (13.4) is calculated via a
straight-forward application of the transform formula (12.19):

$$\begin{align}      \tag*{13.6}
Y(\omega) & =  \int_{-\infty}^{+\infty}dt  \frac{e^{-i\omega
t}}{\sqrt{2\pi}} y(t)    = \frac{-i}{\sqrt{2\pi}}
\int_0^{N\pi/\omega_0} \hspace{-2ex} dt  \sin\omega_0t \sin\omega
t\\
& = \frac{(\omega_0+\omega)\sin\left[(\omega_0-\omega)
\frac{N\pi} {\omega_0}\right]-(\omega_0-\omega)\sin\left[(\omega_0+\omega)
\frac{N\pi}{\omega_0}\right]}{\sqrt{2\pi} (\omega_0^2-\omega^2)},\end{align}$$

where we have dropped a factor of $-i$ that affects only the phase.
While at first glance (13.6) appears to be singular at
$\omega=\omega_0$, it actually just peaks there (Figure 13.3 B),
reflecting the predominance of frequency $\omega_0$. Note that although
the signal $y(t)$ appears to have only one frequency, it does drop off
sharply in time (Figure 13.3B), and these corners give $Y(\omega)$ a
width $\Delta \omega$.

|Figure 13.3 A |Figure 13.3 B| 
|:- - -:|:- - -:| 
|![image](Figs/Fig13_3a.png) |![image](Figs/Fig13_3b.png)|

**Figure 13.3** *A:* A wave packet in time corresponding to the functional
form (13.4) with $\omega_{\text{0}} =
\text{5}$ and $\textit{N} = \text{6}$. *B:* The Fourier transform
in frequency of this same wave packet.

There is a fundamental relation between the widths $\Delta t$ and
$\Delta\omega$ of a wave packet. Although we use a specific example to
determine that relation, it is true in general. While there may not be a
precise definition of “width” for all functions, one can usually deduce
a good measure of the width (say, within 25%). To illustrate, if we look
at the right of Figure 13.3, it makes sense to use the distance between
the first zeros of the transform $Y(\omega)$ (13.6) as the frequency
width $\Delta\omega$. The zeros occur at

$$\tag*{13.7}
\frac{\omega-\omega_0}{\omega_0} = \pm \frac{1}{N} \Rightarrow
\quad \Delta\omega \simeq    \omega-\omega_0    = \frac{\omega_0}{N},$$

where $N$ is the number of cycles in our original wave packet. Because
the wave packet in time makes $N$ oscillations each of period $T$, a
reasonable measure of the time width $\Delta t$ of the signal $y(t)$ is

$$\tag*{13.8}
\Delta t = NT =N \frac{2\pi}{\omega_0}.$$

When the products of the frequency width (13.7) and the time width (13.8) are
combined, we obtain 

$$\begin{align}
\tag*{13.9}
  \Delta t  \Delta\omega \geq 2\pi.\end{align}$$
  
The greater-than sign
is used to indicate that this is a minimum, that is, that $y(t)$ and
$Y(\omega)$ extend beyond $\Delta t$ and $\Delta\omega$, respectively.
Nonetheless, most of the signal and transform should lie within the
bound (13.9).

A relation of the form (13.9) also occurs in quantum mechanics, where it
is known as the *Heisenberg uncertainty principle*, with $\Delta t$ and
$\Delta\omega$ being called the uncertainties in $t$ and $\omega$. It is
true for transforms in general and states that as a signal is made more
localized in time (smaller $\Delta t$) the transform becomes less
localized (larger $\Delta \omega$). Conversely, the sine wave $y(t)=
\sin\omega_0 t$ is completely localized in frequency, and consequently
has an infinite extent in time, $\Delta t \simeq \infty$.

### 13.3.1  Wave Packet Assessment<a id="13.3.1"></a>

Consider the following wave packets:

$$\tag*{13.10} y_1(t) = e^{-t^2/2}, \quad
y_2(t) = \sin(8t) e^{-t^2/2}, \quad y_3(t) = (1-t^2) e^{-t^2/2}.$$

For each wave packet:

1.  Estimate the width $\Delta t$. A good measure might be the *full
    width at half-maxima* (FWHM) of $|y(t)|$.

2.  Use your DFT program to evaluate and plot the Fourier transform
    $Y(\omega)$ for each wave packet. Make *both* a linear and a semilog
    plot (small components are often important, yet not evident in
    linear plots). Make sure that your transform has a good number of
    closely spaced frequency values over a range that is large enough to
    show the periodicity of $Y(\omega)$.

3.  What are the units for $Y(\omega)$ and $\omega$ in your DFT?

4.  For each wave packet, estimate the width $\Delta \omega$. A good
    measure might be the *full width at half-maxima* of $|Y(\omega)|$.

5.  For each wave packet determine approximate value for the constant
    $C$ of the uncertainty principle

    $$\tag*{13.11}
    \Delta t  \Delta\omega \geq 2\pi C.$$

## 13.4   Short-Time Fourier Transforms (Math)<a id="13.4"></a>

The constant amplitude of the functions $\sin n\omega t$ and
$\cos n\omega t$ for all times can limit the usefulness of Fourier
analysis for reproducing signals. Because these functions and their
overtones extend over all times with a constant amplitude, there is
considerable overlap among them, and consequently the information
present in various Fourier components are correlated. This is
undesirable for data storage and compression, where you want to store a
minimum number of information and also want to adjust the amount stored
based on the desired quality of the reconstructed signal\[*Note:*
Wavelets have proven to be a highly effective approach to data
compression, with the Joint Photographic Experts Group (JPEG) 2000
standard being based on wavelets.\]. In *lossless compression*, which
exactly reproduces the original signal, you save space by storing how
many times each data element is repeated, and where each element is
located. In *lossy compression*, in addition to removing repeated
elements, you also eliminate some transform components consistent with
the uncertainty relation (13.9) and with the level of resolution
required in the reproduction. This leads to yet greater compression.

In §12.5 we defined the Fourier transform $Y(\omega)$ of signal $y(t)$
as

$$\tag*{13.12} Y(\omega) = \int_{-\infty}^{+\infty} dt \frac{e^{-i\omega
t}}{\sqrt{2\pi}} y(t) \equiv \left\langle{\omega}\left|{y}\right.\right\rangle .$$

As is true for simple vectors, you can think of (13.12) as giving the
overlap or scalar product of the basis function
$\exp(i\omega t)/\sqrt{2\pi}$ and the signal $y(t)$ \[notice that the
complex conjugate of the exponential basis function appears in
(13.12)\]. Another view of (13.12) is as the mapping or projection of
the signal $y$ into $\omega$ space. In this latter case the overlap
projects out the amount of the periodic function $\exp(i\omega
t)/\sqrt{2\pi}$ in the signal $y(t)$. In other words, the Fourier
component $Y(\omega)$ is also the correlation between the signal $y(t)$
and the basis function $\exp(i\omega t)/\sqrt{2\pi}$, which is the same
as what results from filtering the signal $y(t)$ through a
frequency-$\omega$ filter. If there is no $\exp(i\omega t)$ in the
signal, then the integral vanishes and there is no output. If
$y(t) =\exp(i\omega t)$, the signal is at only one frequency, and the
integral is accordingly singular.

The signal in Figure 13.1 for our problem clearly has different
frequencies present at different times and for different lengths of
time. In the past this signal might have been analyzed with a precursor
of wavelet analysis known as the *short-time Fourier transform*. With
that technique, the signal $y(t)$ is “chopped up” into different
segments along the time axis, with successive segments centered about
successive times $\tau_1,
\tau_2, \ldots, \tau_N$. For instance, we show three such segments in
the boxes of Figure 13.1. Once we have the dissected signal, a Fourier
analysis is made for each segment. We are then left with a sequence of
transforms $\left(Y^{{\rm (ST)}}_{\tau_1},
Y^{{\rm (ST)}}_{\tau_2},\ldots, Y^{{\rm (ST)}}_{\tau_N}\right)$, one for
each short-time interval, where the superscript $\mbox{}^{{\rm (ST)}}$
indicates short time.

Rather than chopping up a signal, we can express short-time Fourier
transforming mathematically by imagining translating a *window function*
$w(t-\tau)$, which is zero outside of some chosen interval, over the
signal in Figure 13.1:

$$\tag*{13.13} Y^{{\rm (ST)}}(\omega,\tau) = \int_{-\infty}^{+\infty}dt
\frac{e^{i\omega t}}{\sqrt{2\pi}}  w(t-\tau)  y(t).$$

Here the values of the translation time $\tau$ correspond to different
locations of the window $w$ over the signal, and the window function is
essentially a transparent box of small size on an opaque background. Any
signal within the width of the window is transformed, while the signal
lying outside the window is not seen. Note that in (13.13), the extra
variable $\tau$ in the Fourier transform indicates the location of the
time around which the window was placed. Clearly, because the short-time
transform is a function of two variables, a surface or 3-D plot is
needed to view the amplitude as a function of both $\omega$ and $\tau$.

## 13.5  The Wavelet Transform<a id="13.5"></a>

[![image](Figs/RHLlectureMod4.png)](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Modules/CWavelet_II/CWavelet_II.html)

The wavelet transform of a time signal $y(t)$ is defined as 

$$\tag*{13.14}
 Y(s, \tau) = \int_{-\infty}^{+\infty}\,dt\, \psi_{s,\tau}^*(t)\, y(t) \quad \mbox{(wavelet transform),}$$

and is similar in concept and notation to a short-time Fourier
transform. The difference is rather than using $\exp(i\omega t)$ as the
basis functions, here we are using wave packets or wavelets
$\psi_{s,\tau}(t)$ localized in time, such as the those shown in
Figure 13.2. Because each wavelet is localized in time, each acts as its
own window function. Because each wavelet is oscillatory, each contains
its own small range of frequencies.

Equation (13.14) says that the wavelet transform $Y(s, \tau)$ is a
measure of the amount of basis function $\psi_{s,\tau}(t)$ present in
the signal $y(t)$. The $\tau$ variable indicates the time portion of the
signal being decomposed, while the $s$ variable is equivalent to the
frequency present during that time:

$$\tag*{13.15}
\omega = \frac{2\pi} {s}, \quad s = \frac{2\pi}{\omega}\quad
\mbox{(scale-frequency relation).}$$

Because it is key to much that follows, it is a good idea to think about
(13.15) for a while. If we are interested in the time *details* of a
signal, then this is another way of saying that we are interested in
what is happening at small values of the *scale* $s$. Equation (13.15)
indicates that small values of $s$ correspond to high-frequency
components of the signal. That being the case, the time details of the
signal are in the high-frequency, or low-scale, components.

### 13.5.1  Generating Wavelet Basis Functions<a id="13.5.1"></a>

The conceptual discussion of wavelets is over, and it is time to get
down to work. We first need a technique for generating wavelet basis
functions, and then we need to discretize this technique. As is often
the case, the final formulation will turn out to be simple and short,
but it will be a while before we get there.

|A|B| 
|:- - -:|:- - -:|
|![image](Figs/Fig13_4a.png)|![image](Figs/Fig13_4b.png)|
|![image](Figs/Fig13_4c.png)|![image](Figs/Fig13_4d.png)|

**Figure 13.4** Four wavelet basis functions (daughters) generated by scaling
($s$) and translating ($\tau$) an oscillating Gaussian mother wavelet. *From
top:* ($\textit{s} = \text{1}$, $\tau =
\text{0}$), ($\textit{s} = \text{1/2}$, $\tau = \text{0}$),
($\textit{s} = \text{1}$, $\tau = \text{6}$), and ($\textit{s}
= \text{2}$, $\tau = \text{60}$). Note how $\textit{s} <
\text{1}$ is a wavelet with higher frequency, while $\textit{s}
> \text{1}$ has a lower frequency than the $\textit{s} = \text{1}$
mother. Likewise, the $\tau = \text{6}$ wavelet is just a translated
version of the $\tau = \text{0}$ one directly above it.

Just as the expansion of an arbitrary function in a complete set of orthogonal
functions is not restricted to any particular set, so too is the wavelet transform
not restricted to any particular wavelet basis, although some might be better
than others for a given signal. The standard way to generate a family of wavelet
basis functions starts with $\Psi(t)$, a *mother* or *analyzing* function of the
real variable $t$, and then to use this to generate *daughter* wavelets. As a
case in point, we start with the mother wavelet 

$$\begin{align}
\tag*{13.16}
\Psi (t) = \sin(8t) e^{-t^2/2}.\end{align}$$ By scaling, translating
and normalizing this mother wavelet,

$$\tag*{13.17}
\psi_{s,\tau}(t)  =  \frac{1}{\sqrt{s}}
\Psi\left(\frac{t-\tau}{s}\right) = \frac{1}{\sqrt{s}} \sin\left[
\frac{8(t-\tau)}{s}\right]  e^{-{(t-\tau)^2} /{2s^2}}.$$


![image](Figs/Fig13_7.png)

**Figure 13.5** A schematic representation of the steps followed in performing
a wavelet transformation over all time displacements and scales. The red
(upper) signal is first analyzed by evaluating its overlap with a narrow wavelet at
the signal’s beginning. This produces a coefficient that measures the similarity
of the signal to the wavelet. The wavelet is successively shifted over the length
of the signal and the overlaps are successively evaluated. After the entire signal
is covered, the wavelet is expanded and the entire analysis is repeated.

we generate the four wavelet basis functions (daughters) displayed in
Figure 13.4. We see that larger or smaller values of $s$, respectively,
expand or contract the mother wavelet, while different values of $\tau$
shift the center of the wavelet. Because the wavelets are inherently
oscillatory, the scaling leads to the same number of oscillations
occurring in different time spans, which is equivalent to having basis
states with differing frequencies. We see that $s<1$ produces a
higher-frequency wavelet, while $s>1$ produces a lower-frequency one,
both of the same shape. As we shall see, we do not need to store much
information to outline the large-time-scale $s$ behavior of a signal
(its *smooth envelope*), but we do need more information to specify its
short-time-scale $s$ behavior (*details*). And if we want to resolve yet
finer features in the signal, then we will need to have more information
on yet finer details. Here the division by $\sqrt{s}$ is made to ensure
that there is equal “power” (or energy or intensity) in each region of
$s$, although other normalizations can also be found in the literature.
After substituting in the definition of daughters, the wavelet transform
(13.14) and its inverse \[[van den Berg(99)](BiblioLinked.html#van)\] are

$$\begin{align}\tag*{13.18}
 Y(s, \tau) &= \frac{1}{\sqrt{s}} \int_{-\infty}^{+\infty}dt
\Psi^*\left(\frac{t-\tau}{s}\right)  y(t)\quad \mbox{(Wavelet TF)},\\
  y(t) &= \frac{1}{C} \int_{-\infty}^{+\infty}
d\tau\int_{0}^{+\infty}ds \frac{\psi^*_{s,\tau}(t)}{s^{3/2}}
Y(s, \tau)  \quad \mbox{(Inverse Transform)}, \tag*{13.19}
\end{align}$$

where the normalization constant $C$ depends on the wavelet used. In
summary, wavelet bases are functions of the time variable $t$, as well
as of the two parameters $s$ and $\tau$. The $t$ variable is integrated
over to yield a transform that is a function of the time scale $s$
(frequency $2\pi/s$) and window location $\tau$. You can think of scale
as being like the scale on a map (also discussed in §16.5.2 in relation
to fractal analysis) or in terms of *resolution*, as might occur in
photographic images. Regardless of the words, as we see in
[Chapter 16](CP16.ipynb), if we have a fractal, then we have a
self-similar object that looks the same at all scales or resolutions.
Similarly, each wavelet in a set of basis functions is self-similar to
the others, but at a different scale or location.

![image](Figs/Fig13_5.png)

**Figure 13.6** Comparison of an input and reconstituted signal (13.23) using
Morlet wavelets (the curves overlap nearly perfectly). As expected for Fourier
transforms, the reconstruction is least accurate near the endpoints.

The general requirements for a mother wavelet $\Psi$ are
\[[Addison(02)](BiblioLinked.html#addison),[van den Berg(99)](BiblioLinked.html#van)\]

1.  $\Psi(t)$ is real.

2.  $\Psi(t)$ oscillates around zero such that its average is zero:

     $$\tag*{13.20}
    \int_{-\infty}^{+\infty}\Psi(t) dt = 0.$$

3.  $\Psi(t)$ is local, that is, a wave packet, and is
    square-integrable:

    $$\tag*{13.21}
    \Psi(t\rightarrow\infty)  \rightarrow 0\quad \mbox{(rapidly)},
    \quad \int_{-\infty}^{+\infty}|Psi(t)|^2 dt    < \infty.$$

4.  The transforms of low powers of $t$ vanish, that is, the first $p$
    moments:

    $$\tag*{13.22}
\int_{-\infty}^{+\infty}t^0 \Psi(t) dt =
\int_{-\infty}^{+\infty} t^1  \Psi(t) dt = \cdots =
\int_{-\infty}^{+\infty}t^{p-1}  \Psi(t) dt =0.$$

This makes the transform more sensitive to details than to general
shape.

![image](Figs/Fig13_6.png)

**Figure 13.7** The continuous wavelet spectrum obtained by analyzing the
input signal with Morelet wavelets. Observe how at small values of time $\tau$
there is predominantly one frequency present, how a second, higher-frequency
(smaller-scale) component enters at intermediate times, and how at larger
times a still higher-frequency components enter. (Figure courtesy of Z.
Dimcovic.)

As an example of how we use the $s$ and $\tau$ degrees of freedom in a
wavelet transform, consider the analysis of a chirp signal
$y(t)=\sin(60 t^2)$ (Figure 13.7). We see that a slice at the beginning
of the signal is compared to our first basis function. (The comparison
is carried out via the *convolution* of the wavelet with the signal.)
This first comparison is with a narrow version of the wavelet, that is,
at low scale, and yields a single coefficient. The comparison at this
scale continues with the next signal slice, and eventually ends when the
entire signal has been covered (the top row in Figure 13.6). Then in the
second row the wavelet is expanded to larger $s$ values, and comparisons
are repeated. Eventually, the data are processed at all scales and at
all time intervals. The narrow wavelets correspond to a high-resolution
analysis, while the broad wavelets correspond to low resolution. As the
scales get larger (lower frequencies, lower resolution), fewer details
of the time signal remain visible, but the overall shape or gross
features of the signal become clearer.

### 13.5.2  Continuous Wavelet Transform Implementation<a id="13.5.2"></a>

We want to develop some intuition as to what wavelet transforms look
like before going on to apply them in unknown situations and to develop
a discrete algorithm. Accordingly, modify the program you have been
using for the Fourier transform so that it now computes the continuous
wavelet transform.

1.  You will want to see the effect of using different mother wavelets.
    Accordingly, write a method that calculates the mother wavelet for

    -   a Morlet wavelet (13.2),

    -   a Mexican hat wavelet (13.3),

    -   a Haar wavelet (the square wave in Figure 13.2).

2.  Try out your transform for the following input signals and see if
    the results make sense:

    -   A pure sine wave $y(t) = \sin 2\pi t$,

    -   A sum of sine waves
        $y(t) = 2.5\sin 2\pi t + 6\sin 4\pi t + 10\sin 6\pi t$,

    -   The nonstationary signal for our problem (13.1) 
    
$$\tag*{13.23}
        y(t) =\begin{cases}
         \sin 2\pi t, & \mbox{for } \ 0 \leq t \leq 2,\\
         5\sin 2\pi t + 10\sin 4\pi t , & \mbox{for } \ 2 \leq t \leq 8,\\
         2.5\sin 2\pi t + 6\sin 4\pi t + 10\sin 6\pi t, &\mbox{for }\ 8 \leq t \leq 12.\end{cases}$$

-   The half-wave function

$$\tag*{13.24}
        y(t) =\begin{cases}
        \sin \omega t, & \mbox{for } \      0<t< T/2 , \\
             0,                 & \mbox{for } \ T/2 < t < T. \end{cases}$$

3.  $\odot$ Use (13.19) to invert your wavelet transform and compare the
    reconstructed signal to the input signal (you can normalize the two
    to each other). In Figure 13.5 we show our reconstruction.

In Listing 11.1 we give our *continuous wavelet transformation* `CWT.py`
\[[Lang & Forinash(98)](BiblioLinked.html#Lang)\]. Because wavelets, with their transforms in two
variables, are somewhat hard to grasp at first, we suggest that you
write your own code and include a portion that does the inverse
transform as a check. In the next section we will describe the *discrete
wavelet transformation* that makes optimal discrete choices for the
scale and time translation parameters $s$ and $\tau$. Figure 13.6 shows
a surface plot of the spectrum produced for the input signal (13.1) in
Figure 13.1. In realization of our goal, we see predominantly one
frequency at short times, two frequencies at intermediate times, and
three frequencies at longer times.

In [2]:
### CWT.py, Notebook Version

In [ ]:
# CWT.py, Notebook Version

from IPython.display import IFrame
from numpy import *
import numpy as np
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D       

iT =  0.0
fT =  12.0 
W = fT - iT
N =  240
h =  W/N 
noPtsSig =  N
noS =  20
noTau =  90
iTau =  0.
iS =  0.1
tau =  iTau
s =  iS 
# Need *very* small s steps for high frequency if s small;
dTau =  W/noTau
dS =  (W/iS)**(1./noS)
maxY =  0.001
sig =  zeros((noPtsSig), float)      # Signal
xx=[]                                # To plot original signal
signl=[]

def signal(noPtsSig,y,xx,signl):             # Signal function
    t = 0.0
    hs = W/noPtsSig
    t1 = W/6.
    t2 = 4.*W/6.
    for i in range(0, noPtsSig):  
        if  t >= iT  and t <=  t1:  
            y[i] =  sin(2*pi*t)
            signl=signl+[y[i]]
            xx=xx+[i]
        elif t >= t1 and t <=  t2: 
            y[i] = 5.*sin(2*pi*t) + 10.*sin(4*pi*t);
            signl=signl+[y[i]]
            xx=xx+[i]
        elif t >= t2 and t <=  fT: 
            y[i] = 2.5*sin(2*pi*t) + 6.*sin(4*pi*t) + 10.*sin(6*pi*t)
            signl=signl+[y[i]]
            xx=xx+[i]
        else: 
            print("In signal(...) : t out of range.")
            sys.exit(1)
          
        t += hs  
    return xx,signl
xx,signl=signal(noPtsSig, sig,xx,signl)      # Form signal
fig1=plt.figure()                            # Plot original signal
ax1=fig1.add_subplot(2,1,1)  
ax1.plot(xx,signl,'r')
plt.xlabel('t')
plt.title('Original signal')

Yn =  zeros( (noS+1, noTau+1), float)        # Transform    
def morlet(t, s, tau):                                         # Mother see eq. 11.14
    T =  (t - tau)/s
    return sin(8*T) * exp( - T*T/2. )

def transform(s, tau, sig):   # see eq. 11.16, finds wavelet transform
    integral = 0.
    t = iT;                                                                # for time
    for i in range(0, len(sig) ):
        t += h
        integral += sig[i]*morlet(t, s, tau)*h
    return integral / sqrt(s)
       
def invTransform(t, Yn):                            # after tranform computes inverse
    s = iS                                                                # transform
    tau = iTau                                     # inverse transform not normalized
    recSig_t = 0                 
    for i in range (0, noS):
        s *= dS                                             # to scale the graph is s
        tau = iTau                                                          # for tau
        for j in range (0, noTau):
            tau += dTau                 
            recSig_t += dTau*dS *(s**( - 1.5) )* Yn[i, j] * morlet(t, s, tau)
    return recSig_t

print("working, finding transform, count 20")
for i in range( 0, noS):
    s *= dS             # was with *                                        # Scaling
    tau = iT
    print(i)
    for j in range(0, noTau):
        tau += dTau                                                    # Translation
        Yn[i, j] = transform(s, tau, sig)
print("transform found")  
for i in range( 0, noS):
    for j in range( 0, noTau):
        if Yn[i, j] > maxY or Yn[i, j] < - 1 *maxY :
            maxY = abs( Yn[i, j] )                                       # Find max Y       
tau =  iT
s =  iS
print("normalize")      
for i in range( 0, noS):
    s *= dS                             
    for j in range( 0, noTau):
        tau +=   dTau                                                    # Transform
        Yn[i, j] = Yn[i, j]/maxY
    tau = iT
print("finding inverse transform")                                   # Find inverse TF
recSigData =  "recSig.dat"                   
recSig =  zeros(len(sig) )                                          # Same resolution
t =  0.0;
print("count to 10")
kco = 0;            j = 0;              Yinv =  Yn             
for rs in range(0, len(recSig) ):            # with inverse transform
    recSig[rs] = invTransform(t, Yinv)       # find the original signal
    #invtr.x[rs] = 2*rs - N                   # here plots the inverse
    #invtr.y[rs] = 10.*recSig[rs]             # transform
    t += h 
    if kco %24 == 0:
        j += 1
        print(j)                             # just to see how long it takes
    kco += 1  
ax1=fig1.add_subplot(2,1,2)  
ax1.plot(xx,recSig,'r')
plt.xlabel('t')
plt.title(' Recover signal using inverse transform')   
x = list(range(1, noS + 1))                  # to plot  in x axis
y = list(range(1, noTau + 1))                                                        # in y
X,Y = plt.meshgrid(x, y)                       # grid for s and tau

def functz(Yn):                              # Function returns transform
    z = Yn[X, Y]    
    return z
                
Z = functz(Yn)                               # function called
fig = plt.figure()                             # creates figure
ax = Axes3D(fig)                             # plots axis for figure
ax.plot_wireframe(X, Y, Z, color = 'r')      # surface of wireframe in red
ax.set_xlabel('s: scale')                    # label axes
ax.set_ylabel('Tau')
ax.set_zlabel('Transform')

plt.show()

working, finding transform, count 20
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
transform found
normalize
finding inverse transform
count to 10
1
2
3
4
5
6
7
8
9
10


## 13.6  Discrete Wavelet Transforms, Multi Resolution Analysis$\odot$<a id="13.6"></a>

[![image](Figs/RHLlectureMod4.png)](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Modules/DWavelet_III/DWavelet_III.html)

As was true for DFT’s, if a time signal is measured at only $N$ discrete
times,

$$\tag*{13.25} y(t_m) \equiv y_m, \quad m = 1, \ldots, N,$$

then we can determine only $N$ independent components of the transform
$Y$. The trick is to remain consistent with the uncertainty principle as
we compute only the $N$ independent components required to reproduce the
signal. The *discrete wavelet transform* (DWT) evaluates the transforms
with discrete values for the scaling parameter $s$ and the time
translation parameter $\tau$:

$$\begin{align} \tag*{13.26}
\psi_{j,k}(t) &\  = \ \frac{\Psi\left[(t- k2^j)/2^j \right]}{\sqrt{2^j}} \ \equiv \   \frac{\Psi\left(t/ 2^j
-k\right)}{\sqrt{2^j}}\quad \mbox{(DWT)}\\
s & = \ 2^j, \quad \tau = \frac{k}{2^j},
\quad k,\; j =
    0, 1, \ldots.\tag*{13.27}\end{align}$$

Here $j$ and $k$ are integers whose maximum values are yet to be
determined, and we have assumed that the total time interval $T
=1$, so that time is always measured in fractional values. This choice
of $s$ and $\tau$ based on powers of 2 is called a *dyadic grid*
arrangement and will be seen to automatically perform the scalings and
translations at the different time scales that are at the heart of
wavelet analysis.\[*Note:* Note that some references scale down with
increasing $j$, in contrast to our scaling up.\] The discrete wavelet
transform now becomes

$$\tag*{13.28} 
Y_{j,k} = \int_{-\infty}^{+\infty} dt \psi_{j,k}(t) y(t)
\simeq \sum_m \psi_{j,k}(t_m)y(t_m)h\quad \mbox{(DWT),}$$

$$where the discreteness here refers to the wavelet basis set and
\emph{not} the time variable. For an orthonormal wavelet basis,
the inverse discrete transform is then

$$ \tag*{13.29}
y(t) = \sum_{j, k =-\infty}^{+\infty}  Y_{j,k}  \psi_{j,k}(t)\quad
\mbox{(inverse DWT)}.$$

This inversion will exactly reproduce the input signal at the $N$ input
points, but only if we sum over an infinite number of terms
\[Addison(02)\]. Practical calculations will be less exact.

![image](Figs/Fig13_8.png)

**Figure 13.8** A graphical representation of the relation between time and
frequency resolutions (the uncertainty relation). Each box represents an equal
portion of the time-frequency plane but with different proportions of time and
frequency.

[**Listing 13.1  CWT.py**](http://www.science.oregonstate.edu/~rubin/Books/CPbook/Codes/PythonCodes/CWT.py)
 computes a normalized continuous wavelet transform
of the signal data in <span>input</span> (here assigned as a sum of sine
functions) using Morlet wavelets (courtesy of Z. Dimcovic). The discrete wavelet
transform (DWT) in Listing 11.2 is faster and yields a compressed transform but
is less transparent. Notice in (13.26) and (13.28) that we have kept the time
variable $t$ in the wavelet basis functions continuous, despite the fact that $s$
and $\tau$ have been made discrete. This is useful in establishing the
orthonormality of the basis functions,

$$\tag*{13.30}
\int_{-\infty}^{+\infty} dt  \psi^*_{j,k}(t)  \psi_{j',k'}(t) =
\delta_{jj'}  \delta_{kk'},$$

where $\delta_{m,n}$ is the Kronecker delta function. Being normalized
to 1 means that each wavelet basis has “unit energy”; being orthogonal
means that each basis function is independent of the others. And because
wavelets are localized in time, the different transform components have
low levels of correlation with each other. Altogether, this leads to
efficient and flexible data storage.

The use of a discrete wavelet basis makes it clear that we sample the
input signal at the discrete values of time determined by the integers
$j$ and $k$. In general, you want time steps that sample the signal at
enough times in each interval to obtain the desired level of precision.
A rule of thumb is to start with 100 steps to cover each major feature.
Ideally, the needed times correspond to the times at which the signal
was sampled, although this may require some forethought.

Consider Figure 13.8. We measure a signal at a number of discrete times within
the intervals ($k$ or $\tau$ values) corresponding to the vertical columns of
fixed width along the time axis. For each time interval we want to sample the
signal at a number of scales (frequencies or $j$ values). However, as discussed
in §13.3, the basic mathematics of Fourier transforms indicates that the width
$\Delta t$ of a wave packet $\psi(t)$ and the width $\Delta \omega$ of its
Fourier transform $Y(\omega)$ are related by an uncertainty principle

$$\begin{align}
\Delta\omega \Delta t \geq 2\pi.\end{align}$$ 

This relation places a
constraint on the intervals in which can measure times based on the
intervals in which we deduce frequencies. So while we may want a
high-resolution reproduction of our signal, we do not want to store more
data than are needed to obtain that reproduction. If we sample the
signal for times centered about some $\tau$ in an interval of width
$\Delta \tau$ (Figure 13.8) and then compute the transform at a number
of scales $s$ or frequencies $\omega=2\pi/s$ covering a range of height
$\Delta \omega$, then the relation between the height and width is
restricted by the uncertainty relation, which means that each of the
rectangles in Figure 13.8 has the same area
$\Delta\omega \Delta t = 2\pi$. The increasing heights of the rectangles
at higher frequencies means that a larger range of frequencies should be
sampled as the frequency increases. The premise here is that the
low-frequency components provide the gross or *smooth* outline of the
signal which, being smooth, does not require much detail, while the
high-frequency components give the details of the signal over a short
time interval and so require many components in order to record these
details with high resolution.

Industrial-strength wavelet analyses do not compute explicit integrals,
but instead apply a technique known as *multiresolution analysis* (MRA)
\[Mallat(89)\]. We give an example of this technique in Figure 13.9 and
in the code `DWT.py` in Listing 13.2. It is based on a *pyramid
algorithm* that samples the signal at a finite number of times, and then
passes it successively through a number of *filters*, with each filter
representing a digital version of a wavelet.

![image](Figs/Fig13_9.png)

**Figure 13.9** A multifrequency dyadic (power-of-2) filter tree used for
discrete wavelet transformations. The L boxes represent lowpass filters and the
H boxes represent highpass filters. Each filter performs a convolution
(transform). The circles containing “$\downarrow\!\text{2}$” filter out half of
the signal that enters them, which is called *subsampling* or *factor-of-2
decimation*. The signal on the left yields a transform with a single low and two
high components (less information is needed about the low components for a
faithful reproduction).

Filters were discussed in §12.8, where in (12.67) we defined the action
of a linear filter as a convolution of the filter response function with
the signal. A comparison of the definition of a filter to the definition
of a wavelet transform (13.14) shows that the two are essentially the
same. Such being the case, the result of the transform operation is a
weighted sum over the input signal values, with each weight the product
of the integration weight times the value of the wavelet function at the
integration point. Therefore, *rather than tabulate explicit wavelet
functions, a set of filter coefficients is all that is needed for
discrete wavelet transforms*.

Because each filter in Figure 13.9 changes the relative strengths of the
different frequency components, passing the signal through a series of
filters is equivalent, in wavelet language, to analyzing the signal at
different scales. This is the origin of the name “multiresolution
analysis.” Figure 13.9 shows how the pyramid algorithm passes the signal
through a series of highpass filters (H) and then through a series of
lowpass filters (L). Each filter changes the scale to that of the level
below. Notice too, the circles containing $\downarrow\!2$ in
Figure 13.9. This operation filters out half of the signal and so is
called *subsampling* or *factor-of-2 decimation*. It is the way we keep
the areas of each box in Figure 13.8 constant as we vary the scale and
translation times. We consider subsampling further when we discuss the
pyramid algorithm.

In summary, the DWT process decomposes the signal into *smooth*
information stored in the low-frequency components and *detailed*
information stored in the high-frequency components. Because
*high-resolution* reproductions of signals require more information
about details than about gross shape, the pyramid algorithm is an
effective way to compress data while still maintaining high resolution.
In addition, because components of different resolutions are independent
of each other, it is possible to lower the number of data stored by
systematically eliminating higher-resolution components. And finally,
use of wavelet filters builds in progressive scaling, which is
particularly appropriate for fractal-like reproductions.

In [ ]:
### DWT.py, Notebook Version

In [ ]:
# DWT.py, Notebook Version, Discrete Wavelet TF, Daubechies type

from IPython.display import IFrame
from numpy import *
import numpy as np
import matplotlib.pyplot as plt

sq3 = sqrt(3)
fsq2 = 4.0*sqrt(2)
N = 1024                           # N = 2^n 
c0 = (1+sq3)/fsq2
c1 = (3+sq3)/fsq2                  # Daubechies 4 coefficents
c2 = (3-sq3)/fsq2
c3 = (1-sq3)/fsq2
f = zeros( (N + 1), float) 

#transfgr1 = None # indicate that displays have not been made yet
#transfgr1 = None

def chirp( xi):                                                        # chirp signal
    y = sin(60.0*xi**2);
    return y;
                  # data vector
inxi = 1.0/N                                 # for chirp signal
xi = 0.0
for i in range(1, N + 1):
    f[i] = chirp(xi)                         # Function to TF
    xi   += inxi;  

def daube4(f, n, sign):        # DWT if sign >= 0, inverse if sign < 0
    global tr,yl,xl,xh,yh
    tr = zeros( (n + 1), float)      # temporary variable
    yl = zeros( (N + 1), float)        # for low pass filter
    xl = zeros( (N + 1), float) 
    yh = zeros( (N + 1), float)        # for high pass filter
    xh = zeros( (N + 1), float) 
    if n < 4 : return
    mp = int(n/2)                    # midpoint of array
    mp1 = mp + 1                     # midpoint plus one
    if sign >= 0:                                                               # DWT
        j = 1
        i = 1
        maxx  = n/2
        while j <= n - 3:
            tr[i] = c0*f[j] + c1*f[j+1] + c2*f[j+2] + c3*f[j+3] # low-pass 
            xl[i]=i
            yl[i]=tr[i]
            tr[i+mp] = c3*f[j] - c2*f[j+1] + c1*f[j+2] - c0*f[j+3]  # high-pass
            xh[i+mp]=i+mp
            yh[i+mp]=tr[i+mp]
            i += 1                                  # d coefficents
            j += 2                                  # downsampling
            #jj+=1
        tr[i] = c0*f[n-1] + c1*f[n] + c2*f[1] + c3*f[2] # low-pass filter
        tr[i+mp] = c3*f[n-1] - c2*f[n] + c1*f[1] - c0*f[2] # high-pass filter
    else:                                               # inverse DWT
        tr[1] = c2*f[mp] + c1*f[n] + c0*f[1] + c3*f[mp1] # low- pass filter
        tr[2] = c3*f[mp] - c0*f[n] + c1*f[1] - c2*f[mp1]# high-pass filter
        j = 3
        for i in range (1, mp):
            tr[j] = c2*f[i] + c1*f[i+mp] + c0*f[i+1] + c3*f[i+mp1]         # low- pass
            j += 1                                                          # upsample
            tr[j] = c3*f[i] - c0*f[i+mp] + c1*f[i+1] - c2*f[i+mp1]         # high-pass
            j += 1;                                     # upsampling
    for i in range(1, n+1):
        f[i] = tr[i]       

def pyram(f, n, sign):                       # DWT, replaces f by TF
    if (n < 4): return                       # too few data
    nend = 4                                 # indicates when to stop
    if sign >= 0 :                           # Transform
        nd = n
        while nd >= nend:                    # Downsample filtering 
            daube4(f, nd, sign)
           
            nd //= 2  
            if nd==512:
                fig=plt.figure()
                ax1=fig.add_subplot(2,1,1) 
                ax1.bar(xl,yl,0.1)
                plt.title('Low pass filter nd=1024/2 ')
               
                ax2=fig.add_subplot(2,1,2) 
                ax2.bar(xh,yh,0.1)
                plt.tight_layout()
                plt.xlabel('x')
                plt.title('High pass filter')
            if nd==32:
                fig1=plt.figure()
                ax3=fig1.add_subplot(2,1,1) 
                ax3.bar(xl,yl,0.1)
                plt.title('Low pass filter nd=32')
               
                ax4=fig1.add_subplot(2,1,2) 
                ax4.bar(xh,yh,0.1)
                plt.tight_layout()
                plt.xlabel('x')
                plt.title('High pass filter')
                
            if nd==8:
                fig2=plt.figure()
                ax5=fig2.add_subplot(2,1,1) 
                ax5.bar(xl,yl,0.1)
                plt.title('Low pass filter nd=8 ')
               
                ax6=fig2.add_subplot(2,1,2) 
                ax6.bar(xh,yh,0.1)
                plt.tight_layout()
                plt.xlabel('x')
                plt.title('High pass filter')  
               
    else:                                    # Inverse TF
        for nd in range (4, n + 1):          # Upsampling
            daube4(f , nd, sign)    
n = N                                        # must be 2^m
pyram(f, n, 1)  

# TF , undelete for inverse tf
# pyram(f, n,  - 1)                          # Inverse TF
plt.show()

### 13.6.1  Pyramid Scheme Implementation $\odot$

We now implement the pyramid scheme outlined in Figure 13.9. The $H$ and
$L$ filters will be represented by matrices, which is an approximate way
to perform the integrations or convolutions. Then there is a decimation
of the output by one-half, and finally an interleaving of the output for
further filtering. This process simultaneously cuts down on the number
of points in the data set and changes the scale and the resolution. The
decimation reduces the number of values of the remaining signal by one
half, with the low-frequency part discarded because the details are in
the high-frequency parts.

![image](Figs/Fig13_10.png)

**Figure 13.10** An input signal at the top is processed by a tree of high- and
low-band filters. The outputs from each filtering are downsampled with half the
data kept. The process continues until there are only two data of high-band
filtering and two data of low-band filtering. When complete, the total number of
output data equals the total number of signal data. The process of wavelet
analysis is thus equivalent to a series of filterings.

As indicated in Figure 13.10, the pyramid DWT algorithm follows five
steps:

1.  Successively applies the (soon-to-be-derived) $c$ matrix (13.41) to
    the whole $N$-length vector,

    $$\tag*{13.31}
    \begin{pmatrix}
    Y_0\
    Y_1\
    Y_2\
    Y_3
    \end{pmatrix} =\begin{pmatrix}
        c_0 &c_1&c_2&c_3\
        c_3&-c_2&c_1&-c_0\
        c_2&c_3&nc_0&c_1\
        c_1&n-c_0&c_3&-c_2
        \end{pmatrix}\begin{pmatrix}
        y_0\
        y_1\
        y_2\
        y_3 \end{pmatrix}.$$

2.  Applies it to the $N/2$-length smooth vector.

3.  Repeats the application until only two smooth components remain.

4.  After each filtering, the elements are ordered, with the newest two
    smooth elements on top, the newest detailed elements below, and the
    older detailed elements below that.

5.  The process continues until there are just two smooth elements left.

To illustrate, here we filter and reorder an initial vector of length
$N=8$:

$$\tag*{13.32}
\begin{pmatrix}
y_1\ y_2\ y_3\ y_4\ y_5\ y_6\ y_7\ y_8
\end{pmatrix} \stackrel{\mbox{filter}}{\longrightarrow}\begin{pmatrix}
s^{(1)}_1\ d^{(1)}_1\ s^{(1)}_2\ d^{(1)}_2\ s^{(1)}_3\ d^{(1)}_3\ s^{(1)}_4\
d^{(1)}_4
\end{pmatrix}\stackrel{\mbox{order}}{\longrightarrow}\begin{pmatrix}
s^{(1)}_1\ s^{(1)}_2\ s^{(1)}_3\
\underline{s^{(1)}_4}\
d^{(1)}_1\ d^{(1)}_2\ d^{(1)}_3\ d^{(1)}_4
\end{pmatrix} \stackrel{\mbox{filter}}{\longrightarrow}\begin{pmatrix}
s^{(2)}_1\ d^{(2)}_1\ s^{(2)}_2\
\underline{d^{(2)}_2}\
d^{(1)}_1\ d^{(1)}_2\ d^{(1)}_3\ d^{(1)}_4
\end{pmatrix} \stackrel{\mbox{order}}{\longrightarrow}\begin{pmatrix}
s^{(2)}_1\ s^{(2)}_2\ d^{(2)}_1\
\underline{d^{(2)}_2}\
d^{(1)}_1\ d^{(1)}_2\ d^{(1)}_3\ d^{(1)}_4
\end{pmatrix}.$$

The discrete inversion of a transform vector back to a signal vector is
made using the transpose (inverse) of the transfer matrix at each stage.
For instance,

$$\tag*{13.33}
\begin{pmatrix}
y_0\ y_1\ y_2\ y_3
\end{pmatrix} =\begin{pmatrix}
    c_0&c_3&c_2&c_1\\
    c_1&-c_2&c_3&-c_0\\
    c_2&c_1&c_0&c_3\\
    c_3&-c_0&c_1&-c_2
\end{pmatrix}\begin{pmatrix}
Y_0\ Y_1\ Y_2\ Y_3
\end{pmatrix}.$$

As a more realistic example, imagine that we have sampled the chirp
signal $y(t) = \sin(60 t^2)$ for 1024 times. The filtering process
through which we place this signal is illustrated as a passage from the
top to the bottom in Figure 13.10. First the original 1024 samples are
passed through a single low band and a single high band (which is
mathematically equivalent to performing a series of convolutions). As
indicated by the down arrows, the output of the first stage is then
downsampled, that is, the number is reduced by a factor of 2. This
results in 512 points from the high-band filter as well as 512 points
from the low-band filter. This produces the first-level output. The
output coefficients from the high-band filters are called
$\{d_i^{(1)}\}$ to indicate that they show details, and $\{s_i^{(1)}\}$
to indicate that they show smooth features. The superscript indicates
that this is the first level of processing. The detail coefficients
$\{d^{(1)}\}$ are stored to become part of the final output.

![image](Figs/Fig13_11.png)

**Figure 13.11** In successive passes, the filtering of the original signal at the
top goes through the pyramid algorithm and produces the outputs shown. The
sampling is reduced by a factor of 2 in each step. Note that in the upper graphs
we have connected the points to emphasize their continuous nature while in the
lower graphs we plot the individual output points as histograms.

In the next level down, the 512 smooth data $\{s_i^{(1)}\}$ are passed
through new low- and high-band filters using a broader wavelet. The 512
outputs from each are downsampled to form a smooth sequence
$\{s_i^{(2)}\}$ of size 256 and a detailed sequence $\{d_i^{(2)}\}$ of
size 256. Again the detail coefficients $\{d^{(2)}\}$ are stored to
become part of the final output. (Note that this is only half the size
of the previously stored details.) The process continues until there are
only two numbers left for the detail coefficients and two numbers left
for the smooth coefficients. Because this last filtering is carried out
with the broadest wavelet, it is of the lowest resolution and therefore
requires the least information.

In Figure 13.11 we show the actual effects on the chirp signal of
pyramid filtering for various levels in the processing. (The processing
is carried out with four-coefficient *Daub4* wavelets, which we will
discuss soon.) At the uppermost level, the wavelet is narrow, and so
convoluting this wavelet with successive sections of the signal results
in smooth components that still contain many large high-frequency parts.
The detail components, in contrast, are much smaller in magnitude. In
the next stage, the wavelet is dilated to a lower frequency and the
analysis is repeated *on just the smooth (low-band) part.* The resulting
output is similar, but with coarser features for the smooth coefficients
and larger values for the details. Note that in the upper graphs we have
connected the points to make the output look continuous, while in the
lower graphs, with fewer points, we have plotted the output as
histograms to make the points more evident. Eventually the downsampling
leads to just two coefficients output from each filter, at which point
the filtering ends.

To reconstruct the original signal (called *synthesis* or
*transformation*) a reversed process is followed: Begin with the last
sequence of four coefficients, upsample them, pass them through low- and
high-band filters to obtain new levels of coefficients, and repeat until
all the $N$ values of the original signal are recovered. The inverse
scheme is the same as the processing scheme (Figure 13.10), only now the
directions of all the arrows are reversed.

### 13.6.2  Daubechies Wavelets via Filtering<a id="13.6.2"></a>

We should now be able to understand that digital wavelet analysis has
been standardized to the point where classes of wavelet basis functions
are specified not by their analytic forms, but rather by their *wavelet
filter coefficients*. In 1988, the Belgian mathematician Ingrid
Daubechies discovered an important class of such filter coefficients
\[[Daubechies(95)](BiblioLinked.html#daub),[Rowe & Abbott(95)](BiblioLinked.html#rowe)\]. We will study just the Daub4 class
containing the four coefficients $c_0$, $c_1$, $c_2$, and $c_3$.

Imagine that our input contains the four elements $\{y_1, y_2, y_3, y_4\}$
corresponding to measurements of a signal at four times. We represent a
lowpass filter $L$ and a highpass filter $H$ in terms of the four filter coefficients
as 

$$\begin{align} \tag*{13.34}
L & =\begin{pmatrix}
c_0 &+c_1&c_2&+c_3
\end{pmatrix}\\
H & = \begin{pmatrix}
 c_3&-c_2&nnc_1&-c_0
 \end{pmatrix}.\end{align}$$ 
 
To see how this works, we form an input
vector by placing the four signal elements in a column and then multiply
the input by $L$ and $H$:

$$\begin{align} 
L\begin{pmatrix} y_0\\ y_1\\ y_2\\ y_3
\end{pmatrix} &=\begin{pmatrix}
 +c_0 &+c_1&+c_2&+c_3
 \end{pmatrix}\begin{pmatrix}
 y_0\\
 y_1\\
 y_2\\
 y_3
 \end{pmatrix}
  =  c_0 y_0+c_1 y_1+c_2 y_2+c_3 y_3,\\
 H\begin{pmatrix}
 y_0\\
 y_1\\
 y_2\\
 y_3
 \end{pmatrix}
 &=\begin{pmatrix}
    +c_3&-c_2&+c_1&-c_0
    \end{pmatrix}\begin{pmatrix}
    y_0\\
    y_1\\
    y_2\\
    y_3
    \end{pmatrix}
    = c_3 y_0-c_2y_1+c_1 y_2-c_0 y_3 .\end{align}$$

We see that if we choose the values of the $c_i$’s carefully, the result
of $L$ acting on the signal vector is a single number that may be viewed
as a weighted average of the four input signal elements. Because an
averaging process tends to smooth out data, the lowpass filter may be
thought of as a *smoothing filter* that outputs the general shape of the
signal.

In turn, we see that if we choose the $c_i$ values carefully, the result
of $H$ acting on the signal vector is a single number that may be viewed
as the weighted differences of the input signal. Because a differencing
process tends to emphasize the variation in the data, the highpass
filter may be thought of as a *detail* filter that produces a large
output when the signal varies considerably, and a small output when the
signal is smooth.

We have just seen how the individual $L$ and $H$ filters, each
represented by a single row of the filter matrix, outputs one number
when acting upon an input signal containing four elements in a column.
If we want the output of the filtering process $Y$ to contain the same
number of elements as the input (four $y$’s in this case), we just stack
the $L$ and $H$ filters together:

$$\tag*{13.36}
\begin{pmatrix}
Y_0\\ Y_1\\ Y_2\\ Y_3
\end{pmatrix} =\begin{pmatrix}
L\\ H\\ L\\ H
\end{pmatrix}\begin{pmatrix}
y_0\\ y_1\\ y_2\\ y_3
\end{pmatrix}=\begin{pmatrix}
c_0 &c_1&c_2&c_3\\
    c_3&-c_2&c_1&-c_0\\
    c_2&c_3&c_0&c_1\\
    c_1&-c_0&c_3&-c_2
\end{pmatrix}\begin{pmatrix}
y_0\\ y_1\\ y_2\\ y_3
\end{pmatrix}.$$

Of course the first and third rows of the $Y$ vector will be identical,
as will the second and fourth, but we will take care of that soon.

Now we go about determining the values of the filter coefficients $c_i$ by
placing specific demands upon the output of the filter. We start by recalling that
in our discussion of discrete Fourier transforms we observed that a transform is
equivalent to a rotation from the time domain to the frequency domain. Yet we
know from our study of linear algebra that rotations are described by
orthogonal matrices, that is, matrices whose inverses are equal to their
transposes. In order for the inverse transform to return us to the input signal,
the transfer matrix must be orthogonal. For our wavelet transformation to be
orthogonal, we must have the $4\times 4$ filter matrix times its transpose
equal to the identity matrix:

$$\begin{align}
\tag*{13.37}
\begin{pmatrix}
    c_0 &c_1&c_2&c_3\\
    c_3&-c_2&c_1&-c_0\\
    c_2&c_3&c_0&c_1\\
    c_1&-c_0&c_3&-c_2
\end{pmatrix}\begin{pmatrix}
    c_0 & c_3&c_2&nc_1\\
    c_1&-c_2&c_3&-c_0\\
    c_2&c_1&c_0&c_3\\
    c_3&-c_0 & c_1 & -c_2
    \end{pmatrix} & =\begin{pmatrix}
    1&0&0&0\\
    0&1&0&0\\
    0&0&1&0\\
    0&0&0&1
\end{pmatrix},  \\
    \Rightarrow\quad
    c^{2}_{0}+ c^{2}_{1}+ c^{2}_{2} +c^{2}_{3}=1,
\quad
    c_2c_0+c_3c_1 & = 0.
\end{align}$$
    
Two equations in four unknowns
are not enough for a unique solution, so we now include the further
requirement that the detail filter $H=(c_3,  -c_0,  c_1,  -c_2)$ must
output a zero if the input is smooth. We define “smooth” to mean that
the input is constant or linearly increasing:

$$\tag*{13.38}
\begin{pmatrix}
y_0 &y_1 & y_2 & y_3
\end{pmatrix} =\begin{pmatrix}
 1 &1 & 1 & 1
\end{pmatrix} \quad\mbox{or}\quad\begin{pmatrix}
 0 & 1 & 2 & 3
\end{pmatrix}.$$

This is equivalent to demanding that the moments up to order $p$ are zero,
that is, that we have an “approximation of order $p$.” Explicitly, 

$$\begin{align}
H\begin{pmatrix}
 y_0 &y_1 & y_2 & y_3
 \end{pmatrix}
 &=  H
\begin{pmatrix}
 1 &1 & 1 & 1
 \end{pmatrix}
 = H\begin{pmatrix}
 0 & 1 & 2 & 3
 \end{pmatrix} =0,\\
\Rightarrow \quad    c_3-c_2+c_1-c_0 &=  0, \quad 0\times c_3
-1\times c_2+2\times c_1-3\times c_0 =0, \\
 \Rightarrow \quad c_0 = \frac{1+\sqrt{3}}{4\sqrt{2}} & \simeq   0.483, \quad    c_1 = \frac{3+\sqrt{3}}{4\sqrt{2}} \simeq
0.836, \tag*{13.39}\\
 c_2 = \frac{3-\sqrt{3}}{4\sqrt{2}}&\simeq   0.224, \quad c_3 =
\frac{1-\sqrt{3}}{4\sqrt{2}} \simeq -0.129.\tag*{13.40}\end{align}$$

These are the basic Daub4 filter coefficients. They are used to create
larger filter matrices by placing the row versions of $L$ and $H$ along
the diagonal, with successive pairs displaced two columns to the right.
For example, for eight elements,

$$\tag*{13.41}
\begin{pmatrix}
Y_0\\ Y_1\\ Y_2\\ Y_3\\ Y_4\\ Y_5\\ Y_6\\ Y_7
\end{pmatrix} =\begin{pmatrix}
    c_0& c_1& c_2& c_3& 0 &0& 0& 0\\
    c_3 & -c_2& c_1& -c_0& 0&0&0&0\\
    0&0&c_0& c_1& c_2& c_3& 0 &0\\
    0&0&c_3 & -c_2& c_1& -c_0& 0 &0\\
    0&0&0&0&c_0& c_1& c_2& c_3\\
    0&0&0&0&c_3 & -c_2& c_1& -c_0\\
    c_2&c_3& 0&0&0&0&c_0&c_1\\
    c_1&-c_0& 0 & 0 & 0 & 0 & c_3 & -c_2
\end{pmatrix}\begin{pmatrix}
    y_0\\
    y_1\\
    y_2\\
    y_3\\
    y_{4}\\
    y_{5}\\
    y_{6}\\
    y_7
\end{pmatrix}.$$

Note that in order not to lose any information, the last pair on the
bottom two rows is wrapped over to the left. If you perform the actual
multiplications indicated in (13.41), you will note that the output has
successive *smooth* and *detailed* information. The output is processed
with the pyramid scheme.

The time dependences of two Daub4 wavelets is displayed in Figure 13.12.
To obtain these from our filter coefficients, first imagine that an
elementary wavelet $y_{1,1}(t) \equiv
\psi_{1,1}(t)$ is input into the filter. This should result in a
transform $Y_{1,1}=1$. Inversely, we obtain $y_{1,1}(t)$ by applying the
inverse transform to a $Y$ vector with a $1$ in the first position and
zeros in all the other positions. Likewise, the $i$<span>th</span>
member of the Daubechies class is obtained by applying the inverse
transform to a $Y$ vector with a $1$ in the $i$<span>th</span> position
and zeros in all the other positions.

|A|B| 
|:- - -:|:- - -:|
|![image](Figs/Fig13_12a.png)|![image](Figs/Fig13_12b.png)|

**Figure 13.12** *A:* The Daub4 e6 wavelet constructed by inverse
transformation of the wavelet coefficients. This wavelet has been found to be
particularly effective in wavelet analyses. *B:* The sum of Daub4 e10 and
Daub4 1e58 wavelets of different scale and time displacements.

On the left in Figure 13.12 is the wavelet for coefficient 6 (thus the
e6 notation). On the right in Figure 13.12 is the sum of two wavelets
corresponding to the coefficients 10 and 58. We see that the two
wavelets have different levels of scale as well as different time
positions. So despite the fact that the time dependence of the wavelets
is not evident when wavelet (filter) coefficients are used, it is there.

### 13.6.3  DWT Implementation and Exercise<a id="13.6.3"></a>

Listing 13.2 gives our program for performing a DWT on the chirp signal
$y(t) = \sin(60t^2)$. The method `pyram` calls the `daube4` method to
perform the DWT or inverse DWT, depending upon the value of `sign`.

1.  Modify the program so that you output to a file the values for the
    input signal that your code has read in. It is always important to
    check your input.

2.  Try to reproduce the left of Figure 13.11 by using various values
    for the variable `nend` that controls when the filtering ends. A
    value `nend=1024` should produce just the first step in the
    downsampling (top row in Figure 13.11). Selecting `nend=512` should
    produce the next row, while `nend=4` should output just two smooth
    and detailed coefficients.

3.  Reproduce the scale-time diagram shown on the right in Figure 13.11.
    This diagram shows the output at different scales and serves to
    interpret the main components of the signal and the time in which
    they appear. The time line at the bottom of the figure corresponds
    to a signal of length 1 over which 256 samples were recorded. The
    low-band (smooth) components are shown on the left, and the
    high-band components on the right.

    -   The bottommost figure results when `nend` $= 256$.

    -   The figure in the second row up results from `nend` $= 128$, and
        we have the output from two filterings. The output contains 256
        coefficients but divides time into four intervals and shows the
        frequency components of the original signal in more detail.

    -   Continue with the subdivisions for `nend` $= 64, 32, 16, 8$,
        and 4.

4.  For each of these choices except the topmost, divide the time by 2
    and separate the intervals by vertical lines.

5.  The topmost spectrum is your final output. Can you see any relation
    between it and the chirp signal?

6.  Change the sign of `sign` and check that the inverse DWT reproduces
    the original signal.

7.  Use the code to visualize the time dependence of the Daubechies
    mother function at different scales.

    -   Start by performing an inverse transformation on the
        eight-component signal `[0,0,0,0,1,0,0,0]`. This should yield a
        function with a width of about 5 units.

    -   Next perform an inverse transformation on a unit vector with
        $N=32$ but with all components except the fifth equal to zero.
        The width should now be about 25 units, a larger scale but still
        covering the same time interval.

    -   Continue this procedure until you obtain wavelets of 800 units.

    -   Finally, with $N=1024$, select a portion of the mother wavelet
        with data in the horizontal interval \[590,800\]. This should
        show some self-similarity.

[**Listing 13.2  DWT.py**](http://www.science.oregonstate.edu/~rubin/Books/CPbook/Codes/PythonCodes/DWT.py) computes the discrete wavelet transform using the
pyramid algorithm for the 2$^{\text n}$ signal values stored in <span>f\[
\]</span> (here assigned as the chirp signal sin 60$\textit{t}^{\text{2}}$). The
Daub4 digital wavelets are the basis functions, and <span>sign =
$\pm$1</span> for transform/inverse.

## 13.7  Principal Components Analysis<a id="13.7"></a> 

We have seen that Fourier analysis has a shortcoming of having all of
its components correlated. This slows down the calculation of transforms
and makes compression and reconstitution of data problematic. Wavelets,
on the other hand, appear excellent at data compression, but not
appropriate for high-dimensionality data sets or for all physical
situations. Principal components analysis (PCA) is excellent for
situations in which there are correlation among the variable in the
data, and especially for the type of space-time correlations as might be
found in brain waves, facial patterns and ocean currents. The same basic
PCA approach is used in many fields with names such as the
Karhunen-Loève transform, the Hotelling transform, the proper orthogonal
decomposition, singular value decomposition, factor analysis, empirical
orthogonal functions, empirical component analysis, empirical modal
analysis, and so forth \[Wiki(14)\].

A key element in PCA is viewing a collection of data as defining a
multi-dimensional *data space*, and then finding a few basic components
in those data that contain most of the "power’. For example, imagine the
output of 32 detectors of magnetic brain waves recorded every tenth of a
second for an hour. In this case there will be 33 space-time dimensions
to the data and $10\times60\times60\times 32 = 1,152,000$ data elements.
And so we are now motivated to transform from the detector-time interval
basis to a new set of basis functions known as the *principal
components* which are pretty much guaranteed to concentrate most of the
signal strength into just a few components \[Jackson(88), Jolliffe(91),
Smith(91)\]. This is analogous to the principal axes theorem of
mechanics, in which the description of the rotation of a solid object is
greatly simplified if moments of inertia relative to the principal axes
are used.

As follows from the example that we will work out soon, on the left of
Figure 13.13 we see a bunch of data point along with the two principal
component eigenvectors for these data. This figure shows how the first
component accounts for most of the variability of the data (has largest
possible variance), while the next component is orthogonal, and thus not
correlated with the first component. The second component accounts for
much less of the data, namely the most possible variance in the data
within the constraint of being orthogonal to the first component.

### 13.7.1  Demonstration of Principal Component Analysis<a id="13.7.1"></a>

The derivation of PCA and proofs of its theorems can get quite involved.
Instead, as our introduction to the subject we take a purely operational
approach and work through a simple PCA analysis following the example of
\[Smith(99)\]. We assume that the data have two dimension, and we call
these $x$ and $y$, but they need not be related to spatial positions.

 **Table 13.1** PCA Data in PCA Basis
 
|** Data** |** Data** |**Adjusted Data** |**Adjusted Data** | ** IN PCA Basis**|** IN PCA Basis**|
|- - -:|- - -:|- - -:|-- -:|- - -:|- - -:| 
|$x$|$y$|$x$|$y$| $x_1$ | $x_2$|
|2.5|2.4 | 0.69| 0.49| -0.828 | -0.175| 
|0.5|0.7|-1.31|-1.21 | 1.78 | 0.143|
|2.2|2.9|0.39|0.99|-0.992 | 0.484| 
|1.9|2.2|0.09|0.29 | -0.274 | 0.130|
|3.1|3.0|1.29|1.09|-1.68 | -0.209|
|2.3|2.7|0.49|0.79 | 0.913 | 0.175|
|2|1.6|0.19|-0.31| 0.0991 | -0.350| 
|1.0|1.1|-0.81|-0.81 | 1.14 | 0.464|
|1.6|1.6|-0.31|-0.31 | 0.438 | 0.0178| 
|1.1| 0.9| -0.71| -1.01 | 1.22 |-0.163|

** 1. Enter Data:** We start with a data set, in our case the first two columns of
Table 13.1.

**2. Subtract the Mean:** PCA analysis assumes that the data in each
dimension has zero mean. Accordingly, as shown in columns two and three in
Table 13.1, we calculate the mean for each column, ($\bar{x}, \bar{y}$), and
then subtract them from the data in each column. The resulting adjusted data
are given in the third and forth columns of the table, and placed here into a data
matrix: $$\tag*{13.42} X =\begin{pmatrix}
 0.69& 0.49\\
  -1.31&-1.21\\
 0.39&0.99\\
 0.09&0.29\\
 1.29&1.09\\
 0.49&0.79\\
 0.19&-0.31\\
 -0.81&-0.81\\
 -0.31&-0.31\\
 -0.71& -1.01\\
\end{pmatrix}.$$

**3. Calculate Covariance Matrix:** Recall that for a data set with $N$
members, the *variance* is a measure of the deviation of the data from their
mean:
$$\tag*{13.43}
\mathrm{var}(x) = \frac{1}{N-1} \sum_{i=1}^N (x_i-\bar{x})^2
 = \frac{1}{N-1}   \sum_{i=1}^N (x_i-\bar{x})(x_i-\bar{x}).$$

When a data set contains multiple dimensions (variables), there may well
be a dependence of the data in one dimensional variable to that in
another. The *covariance* gives a measure of how much the deviation of
one variable from the mean varies with the deviation of another variable
from the mean:

$$\tag*{13.44}
\mathrm{cov}(x,y) = \frac{1}{N-1}  \sum_{i=1}^N (x_i-\bar{x})(y_i-\bar{y}),$$

so that a positive covariance indicates that the $x$ and $y$ variables
tend to change together in the same direction. Also, we see that the
variance (13.43) can be viewed as a special case of the covariance,
$\mathrm{var}(x) = \mathrm{cov}(x,x)$, and that there is a symmetry here
with $\mathrm{cov}(x,y)=\mathrm{cov}(y,x)$. All of these possible
covariance values are combined into a symmetric covariance matrix, which
for our $2\times2$ case is just

$$\tag*{13.45} C =\begin{pmatrix}
\mathrm{cov}(x,x) & \mathrm{cov}(x,y)\\
\mathrm{cov}(y,x)&  \mathrm{cov} (y,y)
\end{pmatrix}.$$

The next step in a PSA is to compute the covariance matrix for all of
the data, which in our case turns out to be

$$\tag*{13.46} C=\begin{pmatrix} 0.6166 & 0.6154\\ 0.6154 & 0.7166
\end{pmatrix}.$$

||| 
|:- - -:|:- - -:| 
|![image](Figs/Fig13_13a.png)|![image](Figs/Fig13_13b.png)|

**Figure 13.13** *Top:* The normalized data and eigenvectors of covariance
matrix. *Bottom:* The normalized data using the PCA eigenvectors as basis.

**4. Compute Unit Eigenvector and Eigenvalues of C:** Easy to do with NumPy:

$$\begin{align}
\tag*{13.47}
\lambda_1 = 1.284,  \qquad & \lambda_2 = 0.4908,\\
\mathbf{e}_1 =\begin{pmatrix}-0.6779\\ -0.7352\end{pmatrix},
 \qquad &
\mathbf{e}_2 =\begin{pmatrix}-0.7352\\0.6789\end{pmatrix} ,\tag*{13.48}\end{align}$$

where we have ordered the eigenvalues and eigenvectors so that the
largest eigenvalue is first. As we shall see, the eigenvector
corresponding to this largest eigenvalue is in fact the principal
component in the data, typically with $\sim$80% of the power in it.

In Figure 13.13 we show the normalized data and the two unit
eigenvectors of the covariance matrix (scaled to fill frame). Notice
that the $\mathbf{e}_1$ eigenvector looks very much like a straight-line
best fit to the data. This is the major trend in the data. The other
eigenvector $\mathbf{e}_2$ is clearly orthogonal to $\mathbf{e}_1$, and
contain much less of the signal strength than $\mathbf{e}_1$. It is in
the direction of the variation from the straight line fit, and cleaerly
contains less information about the data. This are the essential ideas
behind PSA.

**5. Expressing Data in Terms of Principal Components:** Now that we have
the principal components, we need to do something with them, namely, express
the data in terms of them. Clearly, one choice is to ignore the eigenvectors
corresponding to the smaller eigenvalues (only one in our simple example). This
is useful for focusing attention on the key elements in the data, as well as for
compressing the data. What one usually does now is to form a *feature vector*
$F$ made up of the eigenvectors that we wish to keep, for example,

$$\begin{align}
\tag*{13.49}
F_2 & =\begin{pmatrix} -0.6779 & -0.7352\\ -0.7352 & 0.6779\\
\end{pmatrix},  \\
F_1 & =\begin{pmatrix} -0.6779 \\ -0.7352 \\
\end{pmatrix} ,\tag*{13.50}\end{align}$$ 

where $F_1$ keeps just one
principal component, while $F_2$ keeps two. The matrix gets its name
because by deciding which eigenvectors we wish to keep, we are deciding
which features of the data we wish to display.

Next we form the transpose of feature matrix composed of the eigenvectors we
wish to keep, and of the adjusted data matrix:

$$\begin{align}
\tag*{13.51}
F_2^T & =\begin{pmatrix} -0.6779 &-0.7352 \\ -0.7352& 0.6779
\end{pmatrix}, \\
X^T & =\begin{pmatrix} .69& -1.31& .39&.09&1.29& .49& .19&
-.81&-.31&-.71\\ .49&-1.21&.99&.29&1.09&.79&-.31&-.81&-.31& -1.01\\
\end{pmatrix}.\tag*{13.52}\end{align}$$

To express the data in terms of
the principal components, we multiply the transposed feature matrix by the
transposed adjusted data matrix:

$$\begin{align}
\tag*{13.53}
&X^{PCA} = F_2^T \times X^T \\
 &=\begin{pmatrix}
-0.6779 &-0.7352 \\ -0.7352& 0.6779\\
\end{pmatrix}  \tag*{13.54}\\
 &\times\begin{pmatrix}
.69& -1.31& .39&.09&1.29& .49& .19& -.81&-.31&-.71\\
.49&-1.21&.99&.29&1.09&.79&-.31&-.81&-.31& -1.01\\
\end{pmatrix}  \tag*{13.55}\\
& =\begin{pmatrix} .828 & 1.78 & -.992 & -.274 & -1.68 & -.913 & .0991 & 1.15
& .438 & 1.22\\ -.175 & .143 & .384 & .130 & -.209 & .175 & -.350 & .464 &
.178 & -. 162\\
\end{pmatrix}\tag*{13.56}\end{align}$$

In Table 13.1 we place the transform data elements back into standard
form, along side the original data. On the right of Figure 13.13 we show
the normalized data plotted using the eigenvectors $\mathbf{e}_1$ and
$\mathbf{e}_2$ as basis. This plot shows just where each datum point
sits relative to the trend in the data. If we use only the principal
component, we would have all of the data on a straight line (we leave
that as an exercise). Of course our data are so simple that this example
does not show the power of the technique. But if we have millions of
data, it would most valuable to be able to categorize them in terms of a
few components.

### 13.7.2  PCA Exercises<a id="13.7.2"></a>

1.  Use just the principal eigenvector to perform the PCA analysis just
    completed with two eigenvectors.

2.  Store data from ten cycles of the chaotic pendulum studied in
    [Chapter 15](CP15.ipynb), but do not include transients. Perform a
    PCA of these data and plot the results using principal
    component axes.